# Start DASK Client

In [ ]:
from dask.distributed import Client

In [ ]:
option_comp = 'local'
if option_comp == "merlin":
    from dask_jobqueue import SLURMCluster
    cluster = SLURMCluster(cores     = 4, 
                           memory    ='4GB', 
                           walltime  = '10:00:00',
                           interface ='ib0',
                           local_directory = '/data/user/kim_a',
                           log_directory   = '/data/user/kim_a',
                          ) 
    
elif option_comp == "local":
    from dask.distributed import LocalCluster
    cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
n_workers = 4
cluster.scale(n_workers)

In [ ]:
client

In [ ]:
# client.close()
# cluster.close()

# Postprocessing of model outputs Y and saving

In [ ]:
import bw2data as bd
import numpy as np
import pickle
import dask
from pathlib import Path

# Local files
from setups import task_per_worker

if __name__ == '__main__':
    project = 'Geothermal'
    bd.projects.set_current(project)

    option = 'conventional.diff_distributions'
    if "diff_distributions" in option:
        diff_distr = True  # set to true when checking for robustness of GSA results to distribution choice
    else:
        diff_distr = False
    iterations = 5  # number of Monte Carlo iterations per model input

    # Directory for saving results
    write_dir = Path("write_files")
    option_dir = "{}.N{}".format(option, iterations)
    write_dir_option = write_dir / option_dir / "scores"
    write_dir_option.mkdir(parents=True, exist_ok=True)

In [ ]:
# %%time
# # Test
# n_workers = 25
# i_worker = 5
# task_per_worker(project, iterations, option, n_workers, i_worker, write_dir_option, diff_distr=diff_distr)

In [ ]:
task_per_worker = dask.delayed(task_per_worker)

model_evals = []
for i in range(n_workers):
    model_eval = task_per_worker(project, iterations, option, n_workers, i, write_dir_option, diff_distr=diff_distr)
    model_evals.append(model_eval)

In [ ]:
%%time
Y_intermediate = dask.compute(model_evals)  

In [ ]:
Y_intermediate = np.array(Y_intermediate).squeeze()
Y_all_methods = np.vstack(Y_intermediate)

filepath_Y = write_dir_option / 'all_scores.pkl'
with open(filepath_Y, 'wb') as f:
    pickle.dump(Y_all_methods, f)
